# Oefening Lineare Regressie

In deze oefening ga je verder oefenen op het gebruik van Python en een aantal data packages.
Los onderstaande vragen op door code te schrijven in de code cellen.
Zorg ervoor dat deze cellen vanboven naar beneden in het document uitgevoerd kunnen worden (Cell > Run All).
Verwijder ook de output voor het indienen om de bestandsgrootte te beperken (Cell > All Output > Clear). 

Om de dataset te kunnen downloaden met onderstaande code heb je een api-key nodig van Kaggle.
Deze vind je door naar https://www.kaggle.com/ te gaan en rechtsbovenaan op je icoontje te duwen en naar je account-settings te gaan.
Als je een beetje naar beneden scrolt, zie je een knop staan 'Create New API Token'.
Dit download een file waarin je de nodige gegevens vindt om datasets te kunnen downloaden.

Vergeet ook niet te beantwoorden op de vragen over je oplossingen!

De deadline voor deze opgave is: 6 oktober 23:59.

## Opgave

Bij regressie wordt er gezocht om een zo goed mogelijke voorspelling te maken van een continue waarde.
Er moet dus gezocht worden naar een verband tussen de features en de target.
In deze oefening wordt de impact van marketingbudget op de verkoopcijfers bestudeerd.

In [1]:
# imports 
import numpy as np
import pandas as pd 
import opendatasets as od
from zipfile import ZipFile

# Machine Learning tools
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

# Visualisatie
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(color_codes=True)

%matplotlib inline

## Marketingbudget vs Verkoopcijfers

Zoals aangehaald hierboven gaat de eerste oefening over de impact van het marketing budget op de behaalde verkoopcijfers.
De dataset die hiervoor gebruikt kan worden vind je hier: https://www.kaggle.com/ashydv/advertising-dataset.

Begin met in de cel hieronder de dataset te downloaden en in te laden

In [ ]:
od.download("https://www.kaggle.com/ashydv/advertising-dataset")

In [2]:
df_advertising = pd.read_csv(".\\advertising-dataset\\advertising.csv")

### Data Analysis

**Bereken de statische eigenschappen van alle features (max, min, gemiddelde en standaardafwijking). Kan je hieruit bepaalde conclusies trekken?**

Antwoord: Tv reclame heeft veruit het grootste budget. Gemiddelde van radio en krant is gelijkaardig maar de maxima liggen wel sterk uiteen.

In [9]:
stats = df_advertising.describe()
stats

,TV,Radio,Newspaper,Sales
count,200.000000,200.000000,200.000000,200.000000
mean,147.042500,23.264000,30.554000,15.130500
std,85.854236,14.846809,21.778621,5.283892
min,0.700000,0.000000,0.300000,1.600000
25%,74.375000,9.975000,12.750000,11.000000
50%,149.750000,22.900000,25.750000,16.000000
75%,218.825000,36.525000,45.100000,19.050000
max,296.400000,49.600000,114.000000,27.000000


**Zijn er outliers in deze dataset? Dit kan je zien door een groot verschil tussen maximum en 75% waarden (of minimum en 25%)**

Antwoord:

In [8]:
# kan visueel gecontroleerd worden

# of geteld door middel van ze te detecteren
(stats.loc["max"]-stats.loc["75%"]) / stats.loc["max"]

TV           0.261724
Radio        0.263609
Newspaper    0.604386
Sales        0.294444
dtype: float64

### Preprocessing van de dataset

De bovenstaande dataset bevat zowel de features als de target waarden.
Splits nu de dataset in twee zodat de features in een apart object zetten en niet meer samen met je features.
Daarna moeten de features en targets nog verdeeld worden in een trainingsset en testset.
Doe dit hieronder:

In [11]:
y = df_advertising.Sales
X = df_advertising.drop("Sales", axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True)
print(X.shape, X_train.shape)

(200, 3) (150, 3)


Uit de statistische gegevens hierboven zagen we dat de schaal van de features niet gelijk is voor alle features. 
Een normalisatie-stap dringt zich op om dit te corrigeren voor de performantie en accuraatheid van het model. 
Zorg in de cel hieronder dat zowel je trainingsdata en testdata correct genormaliseerd is met gemiddelde 0 en standaardafwijking 1.

In [14]:
scaler = StandardScaler()
# ook bij normalisatie enkel trainingsdata gebruiken en geen testdata
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
# OF
X_train_scaled = scaler.fit_transform(X_train)

# test data wel schalen maar niet in de fit
X_test_scaled = scaler.transform(X_test)
X_train

,TV,Radio,Newspaper
54,262.7,28.8,15.9
124,229.5,32.3,74.2
128,220.3,49.0,3.2
58,210.8,49.6,37.7
64,131.1,42.8,28.9
...,...,...,...
36,266.9,43.8,5.0
37,74.7,49.4,45.7
55,198.9,49.4,60.0
32,97.2,1.5,30.0


### Enkelvoudige regresse

Nu dat de data gepreprocessed is kan het model getrained worden om de verkoopcijfers te voorspellen. 
Doe dit eerst aan de hand van enkelvoudige regressie.
Selecteer hiervoor de feature waarvan je denkt dat die het sterkste verband heeft met de output.
Bereken voor het bekomen model de mean absolute error, mean squared error en $R^2$ score voor zowel de trainings als testset. 
Tip: schrijf hiervoor een functie waaraan je het model en de training en test data meegeeft om het te kunnen hergebruiken.

**Kun je uit deze scores opmaken over er sprake is van overfitting of underfitting van het model?**

Antwoord:R2 score op train en test is niet hoog (minder dan 0.8) dus is er sprake van underfitting (minder dan 0.8)

In [22]:
def regression(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    predictions_train = model.predict(X_train)
    predictions = model.predict(X_test)
    
    print("Train:")
    print("MAE:", mean_absolute_error(predictions_train, y_train))
    print("MSE:", mean_squared_error(predictions_train, y_train))
    print("R2:", r2_score(predictions_train, y_train))
    print("Test:")
    print("MAE:", mean_absolute_error(predictions, y_test))
    print("MSE:", mean_squared_error(predictions, y_test))
    print("R2:", r2_score(predictions, y_test))
    

model = LinearRegression()
# hier kies ik 1 feature ([:, 0] zegt hou alle rijen over en kies kolom 0)
regression(model, X_train_scaled[:,0].reshape(-1, 1), X_test_scaled[:,0].reshape(-1, 1), y_train, y_test)

Train:
MAE: 1.7620058136467114
MSE: 4.910967757563955
R2: 0.7865898176091626
Test:
MAE: 2.0327380444204617
MSE: 6.280776562521073
R2: 0.767204622338012


### Modeloptimalisaties

Na bovenstaande enkelvoudige regressies kunnen we nog een aantal optimalisaties doorvoeren. 
Geef voor elke van de onderstaande mogelijkheden dezelfde scores als bij de enkelvoudige regressie weer en geef weer in het antwoord of er overfitting of underfitting is.

**Meervoudige regressie:** Gebruik all features uit de dataset

Antwoord: Er is geen sprake van overfitting want verschil tussen train en test resultaten zijn klein. Underfitting is nog altijd mogelijk omdat de r2 score nog niet heel hoog is (nog niet boven de 0.9)

In [23]:
model = LinearRegression()
regression(model, X_train_scaled, X_test_scaled, y_train, y_test)

Train:
MAE: 1.1949819679112295
MSE: 2.570093630556764
R2: 0.8986266166923342
Test:
MAE: 1.412351876146451
MSE: 3.2007123896248597
R2: 0.8822475049992767


**Hogere-orde features:** Voeg manueel (zonder gebruik te maken van een functie) hogere-orde features toe aan de dataset. Kies zelf tot welke rang je gaat.

Antwoord: Zowel de score op trainings als testdata is aan het verbeteren dus we gaan de goede kant op. Is er nog geen overfitting want het verschil test en trainingsdata is klein.

In [24]:
# voeg hogere orde features toe
df_enhanced = df_advertising
df_enhanced["TVSquared"] = df_enhanced.TV **2
df_enhanced["TVCube"] = df_enhanced.TV **3
df_enhanced["NewspaperSquared"] = df_enhanced.Newspaper **2
df_enhanced["NewspaperCube"] = df_enhanced.Newspaper **3
df_enhanced["RadioSquared"] = df_enhanced.Radio **2
df_enhanced["RadioCube"] = df_enhanced.Radio **3

# splits targets en features
y2 = df_enhanced.Sales
X2 = df_enhanced.drop("Sales", axis=1)
# verdeel in train en test data
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, shuffle=True)

#normalisatie
scaler = StandardScaler()
X_train_scaled2 = scaler.fit_transform(X_train2)
X_test_scaled2 = scaler.transform(X_test2)

model = LinearRegression()
regression(model, X_train_scaled2, X_test_scaled2, y_train2, y_test2)

Train:
MAE: 1.0459607246146432
MSE: 1.9263653271427323
R2: 0.9234022201773353
Test:
MAE: 1.1784727388994904
MSE: 2.3099846195063507
R2: 0.9190422740743329


In [26]:
df_enhanced
# f(x) = w_0 + w_1 TV + w_2 krant..
# f(x) = w_0 + w_1 TV + w_2 TV^2 + w_3 TV^3 + w_4 krant ...

,TV,Radio,Newspaper,Sales,TVSquared,TVCube,NewspaperSquared,NewspaperCube,RadioSquared,RadioCube
0,230.1,37.8,69.2,22.1,52946.01,1.218288e+07,4788.64,331373.888,1428.84,54010.152
1,44.5,39.3,45.1,10.4,1980.25,8.812112e+04,2034.01,91733.851,1544.49,60698.457
2,17.2,45.9,69.3,12.0,295.84,5.088448e+03,4802.49,332812.557,2106.81,96702.579
3,151.5,41.3,58.5,16.5,22952.25,3.477266e+06,3422.25,200201.625,1705.69,70444.997
4,180.8,10.8,58.4,17.9,32688.64,5.910106e+06,3410.56,199176.704,116.64,1259.712
...,...,...,...,...,...,...,...,...,...,...
195,38.2,3.7,13.8,7.6,1459.24,5.574297e+04,190.44,2628.072,13.69,50.653
196,94.2,4.9,8.1,14.0,8873.64,8.358969e+05,65.61,531.441,24.01,117.649
197,177.0,9.3,6.4,14.8,31329.00,5.545233e+06,40.96,262.144,86.49,804.357
198,283.6,42.0,66.2,25.5,80428.96,2.280965e+07,4382.44,290117.528,1764.00,74088.000


**Hogere-orde features:** Gebruik nu terug alle features en voeg de hogere-orde features toe door tot rang 5 door gebruik te maken van een functie. Hoeveel features worden er dan gebruikt om het target te voorspellen?

Antwoord: In vergelijking met het vorige resultaat is de score van op de trainingsdata verbeterd maar op de testdata is het verslechterd. We zijn dus beginnen overfitten.

In [36]:
higher_order = PolynomialFeatures(5)
X_train_scaled3 = higher_order.fit_transform(X_train_scaled)
X_test_scaled3 = higher_order.transform(X_test_scaled)

model = LinearRegression()
regression(model, X_train_scaled3, X_test_scaled3, y_train, y_test)

Train:
MAE: 0.6656241861979166
MSE: 0.7927714563588301
R2: 0.9711720343018676
Test:
MAE: 1.2878593749999998
MSE: 2.668958458596468
R2: 0.8827904694336216


**Regularisatie:** Voer regularisatie toe aan het model. Ga op zoek naar een goede waarde voor de $\alpha$ of $\lambda$. Probeer dit met zowel de L2-norm als de L1-norm. Welke geeft het beste resulaat?

Antwoord: ...

In [47]:
model = Ridge(alpha=2)
regression(model, X_train_scaled3, X_test_scaled3, y_train, y_test)

Train:
MAE: 0.6634113940756877
MSE: 0.873036290655989
R2: 0.9670694898519597
Test:
MAE: 1.1650853447655345
MSE: 2.613312782648306
R2: 0.8859609371245325


In [43]:
model = Lasso(alpha=0.05)
regression(model, X_train_scaled3, X_test_scaled3, y_train, y_test)

Train:
MAE: 0.7817690291416881
MSE: 1.1345551215670795
R2: 0.956563916537474
Test:
MAE: 1.2287304359158449
MSE: 2.3908448558296937
R2: 0.8980397941720484
